In [1]:
import os, gc
from collections import defaultdict

import pandas as pd
from tqdm.auto import tqdm
import numpy as np

In [2]:
def read_train_terms(path):
    mapping = defaultdict(list)
    df = pd.read_csv(path, sep="\t", header=None, names=["protein","go","ont"], dtype=str)
    for _, r in tqdm(df.iterrows(), total=len(df)): 
        mapping[r.protein].append(r.go)
    print(f"[io] Read training annotations for {len(mapping)} proteins from {path}")
    return mapping

def parse_obo(go_obo_path):
    parents = defaultdict(set)
    children = defaultdict(set)
    
    if not os.path.exists(go_obo_path): 
        return parents, children
        
    with open(go_obo_path,"r") as f:
        cur_id=None
        for line in f:
            line=line.strip()
            if line=="[Term]": 
                cur_id=None
            elif line.startswith("id: "): 
                cur_id=line.split("id: ")[1].strip()
            elif line.startswith("is_a: "):
                pid=line.split()[1].strip()
                if cur_id: 
                    parents[cur_id].add(pid)
                    children[pid].add(cur_id)
            elif line.startswith("relationship: part_of "):
                parts=line.split(); 
                if len(parts)>=3:
                    pid=parts[2].strip()
                    if cur_id: 
                        parents[cur_id].add(pid)
                        children[pid].add(cur_id)
    print(f"[io] Parsed OBO: {len(parents)} nodes with parents")
    return parents, children

def get_ancestors(go_id, parents):
    ans=set()
    stack=[go_id]
    while stack:
        cur=stack.pop()
        for p in parents.get(cur,[]): 
            if p not in ans:
                ans.add(p)
                stack.append(p)
    return ans

def get_descendants(go_id):
    desc = set()
    stack = [go_id]
    while stack:
        cur = stack.pop()
        for child in children_map.get(cur, []):
            if child not in desc:
                desc.add(child)
                stack.append(child)
    return desc


In [ ]:
train_terms = read_train_terms("data/raw/cafa6/Train/train_terms.tsv")
parents_map, children_map = parse_obo("data/raw/cafa6/Train/go-basic.obo")

  0%|          | 0/537028 [00:00<?, ?it/s]

[io] Read training annotations for 82405 proteins from /kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv
[io] Parsed OBO: 40121 nodes with parents


In [ ]:
go_annotations = pd.read_csv('data/processed/uniprot/goa_uniprot_all.csv')
go_annotations = go_annotations.drop_duplicates()
print(f'[+] Dataset shape: {go_annotations.shape}')
go_annotations.head()

[+] Dataset shape: (2583077, 3)


,protein_id,go_term,qualifier
0,A1CHB5,GO:0042407,involved_in
1,A1CHB5,GO:0061617,part_of
2,A1CHB5,GO:0005743,located_in
3,Q9ZFU7,GO:0000987,enables
4,Q9ZFU7,GO:0003700,enables


In [5]:
go_annotations.qualifier.value_counts()

qualifier
involved_in                                       901478
enables                                           791908
located_in                                        487152
acts_upstream_of_or_within                        145517
is_active_in                                      135499
part_of                                            98512
contributes_to                                      7198
acts_upstream_of                                    5207
colocalizes_with                                    3837
NOT|involved_in                                     2078
NOT|enables                                         1919
NOT|located_in                                      1101
acts_upstream_of_or_within_positive_effect           552
acts_upstream_of_positive_effect                     524
acts_upstream_of_negative_effect                     217
acts_upstream_of_or_within_negative_effect           182
NOT|is_active_in                                      81
NOT|part_of          

<p style="background-color:#e6f7ff; 
          padding:15px; 
          color:#111;
          font-size:16px;
          border-width:3px; 
          border-color:#d0eefc; 
          border-style:solid;
          border-radius:6px">
🧬 <code>NOT</code> qualifiers explicitly indicate that a <code>protein</code> is <em>not</em> associated with a given <code>GO term</code>. These annotations serve as reliable <code>negative examples</code>.<br>
If a protein is not linked to a specific <code>GO term</code>, it is also not linked to any of its <code>descendants</code> within the Gene Ontology hierarchy.<br>
By propagating these <code>negative annotations</code> through the ontology, we can reduce <code>false positives</code> and achieve a <strong>more accurate leaderboard score</strong>.
</p>


In [6]:
print(f"[1/3] Filtering Negative Annotations ..")
negative_annots = go_annotations[go_annotations['qualifier'].str.contains('NOT', na=False)]
negative_annots = negative_annots.drop(columns=['qualifier']).drop_duplicates()

print(f"[2/3] Propagate Negative Terms ..")
negative_annots = negative_annots.groupby('protein_id')['go_term'].apply(list).to_dict()

propagated={}
for p in tqdm(negative_annots.keys()):
    terms=set(negative_annots[p])
    extra=set()
    for t in list(terms): 
        extra |= get_descendants(t)
    propagated[p] = sorted(terms | extra)
        
negative_annots = propagated

print(f"[3/3] Extract Unique Keys ..")
rows = [(protein_id, go_term) for protein_id, terms in negative_annots.items() for go_term in terms]
negative_df = pd.DataFrame(rows, columns=["protein_id", "go_term"])
negative_df['pred_key'] = negative_df.protein_id.apply(str) + '_' + negative_df.go_term.apply(str)
negative_keys = set(negative_df['pred_key'])

del negative_df
gc.collect()

print(f"Total unique negative protein-GO pairs: {len(negative_keys)}")

[1/3] Filtering Negative Annotations ..
[2/3] Propagate Negative Terms ..


  0%|          | 0/3863 [00:00<?, ?it/s]

[3/3] Extract Unique Keys ..
Total unique negative protein-GO pairs: 238553


In [ ]:
print(f"[1/4] Loading GOA Annotations ..")
go_annotations = pd.read_csv('data/processed/uniprot/goa_uniprot_all.csv')

print(f"[2/4] Removing unwanted annotations ..")
go_annotations = go_annotations[~go_annotations['qualifier'].str.contains('NOT', na=False)]
go_annotations.drop(columns=['qualifier'], inplace=True)
go_annotations = go_annotations.drop_duplicates()

print(f"[3/4] Set Ground-Truth Score ..")
go_annotations['score'] = round(1.0, 3)

print(f"[4/4] Setting Key ..")
go_annotations['pred_key'] = go_annotations['protein_id'].astype(str) + '_' + go_annotations['go_term'].astype(str)
go_annotations = go_annotations[~go_annotations['pred_key'].isin(negative_keys)]
goa_pred_keys = set(go_annotations['pred_key'])
print(f"[+] Total unique ground truth protein-GO pairs: {len(goa_pred_keys)}")
print(f"[✅] Done.")

[1/4] Loading GOA Annotations ..
[2/4] Removing unwanted annotations ..
[3/4] Set Ground-Truth Score ..
[4/4] Setting Key ..
[+] Total unique ground truth protein-GO pairs: 2462485
[✅] Done.


## Post-processing Submissions

In [18]:
def load_submission(path, chunksize=50000, num_rows = None):
    total = None
    if num_rows:
        total = int(num_rows / chunksize) + 1 
    chunks = []
    for chunk in tqdm(pd.read_csv(path, sep='\t', header=None, chunksize=chunksize), total = total):
        chunk['pred_key'] = chunk[0].astype(str) + '_' + chunk[1].astype(str)
        chunks.append(chunk)
    df = pd.concat(chunks, ignore_index=True)
    df.columns = ['protein_id', 'go_term', 'score', 'pred_key']
    return df

In [ ]:
print(f"Loading submission ..")
submission = load_submission('U900/sub/submission.tsv')
submission.shape

Loading submission ..


0it [00:00, ?it/s]

(9414660, 4)

In [22]:
print(f"Removing Negatives from submission ..")
submission = submission[~submission.pred_key.isin(negative_keys)]
submission.drop(columns=['pred_key'], inplace=True)
submission.shape

Removing Negatives from submission ..


(9408406, 3)

In [ ]:
print(f'[⏳] Saving Submission ...')
submission.to_csv('outputs/submissions/submission.tsv',sep='\t', index=False, header=None)
print(f"[✅] Done.")

[⏳] Saving Submission ...
[✅] Done.


In [ ]:
!head outputs/submissions/submission.tsv

P77503	GO:0008679	0.031
Q9SCX3	GO:0000325	0.419
B1XBX3	GO:1990204	0.145
Q5BK22	GO:0160236	0.028
Q15276	GO:0050794	0.99
Q54ND5	GO:0003674	0.99
A7KTC5	GO:0090567	0.99
Q5B563	GO:0006950	0.99
Q8WWZ7	GO:0017111	0.99
P41743	GO:0048513	0.99
